In [9]:
import pandas as pd
import numpy as np
import pickle

In [10]:
#import csv
data_train = pd.read_csv("../../data/data_train.csv")

In [11]:
#Memilih kolom yang paling berpengaruh pada penilaian
data_train = data_train[["battery_power", "mobile_wt", "px_height", "px_width", "ram", "price_range"]]
data_train

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,804,89,709,818,2027,1
1,1042,139,68,1018,2826,2
2,1481,105,249,522,2635,2
3,1104,199,653,1413,1229,0
4,652,142,464,781,565,0
...,...,...,...,...,...,...
1395,536,135,547,705,1211,0
1396,1097,160,1277,1352,2219,2
1397,1179,182,85,1451,340,0
1398,719,113,431,1727,3990,3


In [12]:
#Mengubah kolom dari nilai numerik menjadi nilai kategorikal
def categorical(dataset):
    batas = [[864.75, 1219.9, 1655.75], [111.0, 143.0, 172.0], [307.0, 576.5, 928.5], [874.0, 1249.0, 1639.75], [1220.75, 2286.0, 3114.5]]
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    for i in range(len(batas)):
        dataset.loc[dataset[nama[i]] <= batas[i][0], nama[i]] = 0
        dataset.loc[(dataset[nama[i]] > batas[i][0]) & (dataset[nama[i]] <= batas[i][1]), nama[i]] = 1
        dataset.loc[(dataset[nama[i]] > batas[i][1]) & (dataset[nama[i]] <= batas[i][2]), nama[i]] = 2
        dataset.loc[dataset[nama[i]] > batas[i][2], nama[i]] = 3
    return dataset

categorical_data_train = categorical(data_train)
categorical_data_train

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,0,0,2,0,1,1
1,1,1,0,1,2,2
2,2,0,0,0,2,2
3,1,3,2,2,1,0
4,0,1,1,0,0,0
...,...,...,...,...,...,...
1395,0,1,1,0,0,0
1396,1,2,3,2,1,2
1397,1,3,0,2,0,0
1398,0,1,1,3,3,3


In [13]:
# kalkulasi P(Vj)
def p_vj(data_train):
    list = []
    for i in np.unique(data_train["price_range"]):
        count = sum(data_train["price_range"] == i)
        list.append(count/len(data_train))
    return list

prior = p_vj(categorical_data_train)
print(prior)

[0.2557142857142857, 0.2542857142857143, 0.24642857142857144, 0.24357142857142858]


In [14]:
# kalkulasi P(Ai|Vj)
def p_ai_vj(data_train):
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    list = []
    for i in range(len(nama)):
        kolom = []
        print(i)
        for j in np.unique(data_train["price_range"]):
            print("price range ", j)
            matrix = []
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 0))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 1))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 2))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 3))
            matrix.append(count/sum(data_train["price_range"] == j))
            kolom.append(matrix)
        list.append(kolom)    
    return list

likelihood = p_ai_vj(categorical_data_train)

# print matrix likelihood
for i in range(len(likelihood)):
    print("matrix likelihood ", i)
    for j in range(len(likelihood[i])):
        print(likelihood[i][j])

0
price range  0
price range  1
price range  2
price range  3
1
price range  0
price range  1
price range  2
price range  3
2
price range  0
price range  1
price range  2
price range  3
3
price range  0
price range  1
price range  2
price range  3
4
price range  0
price range  1
price range  2
price range  3
matrix likelihood  0
[0.31564245810055863, 0.32122905027932963, 0.2346368715083799, 0.12849162011173185]
[0.23876404494382023, 0.26685393258426965, 0.27247191011235955, 0.22191011235955055]
[0.2782608695652174, 0.22028985507246376, 0.2811594202898551, 0.22028985507246376]
[0.16422287390029325, 0.1906158357771261, 0.3343108504398827, 0.31085043988269795]
matrix likelihood  1
[0.2709497206703911, 0.22905027932960895, 0.23743016759776536, 0.26256983240223464]
[0.26685393258426965, 0.23876404494382023, 0.25, 0.2443820224719101]
[0.24347826086956523, 0.2579710144927536, 0.27246376811594203, 0.22608695652173913]
[0.34310850439882695, 0.25806451612903225, 0.22287390029325513, 0.1759530791

Validasi Data Latih

In [15]:
import numpy as np
import pandas as pd

In [16]:
# data_validation = pd.read_csv("../../data/data_validation.csv")
# data_validation = data_validation[["battery_power", "mobile_wt", "px_height", "px_width", "ram", "price_range"]]
# categorical_data_validation = categorical(data_validation)
# categorical_data_validation

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,0,3,0,0,2,1
1,0,2,3,3,2,2
2,0,3,0,0,3,3
3,0,2,2,3,0,0
4,0,1,1,0,3,3
...,...,...,...,...,...,...
595,2,1,1,1,1,1
596,3,1,0,0,3,3
597,0,3,1,3,0,0
598,3,3,2,3,1,2


In [17]:
# # menghitung posterior

def posterior(data_validation, prior, likelihood):
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    list = []
    for i in range(len(data_validation)):
        matrix = []
        for j in range(len(prior)):
            matrix.append(prior[j])
            for k in range(len(nama)):
                matrix[j] = matrix[j] * likelihood[k][j][data_validation[nama[k]][i]]

        list.append(matrix)
    return list

Posterior = posterior(categorical_data_validation, prior, likelihood)


In [18]:
def predict(Posterior, data_validation):
    list = []
    for i in range(len(Posterior)):
        list.append(np.argmax(Posterior[i]))
    count = 0
    for i in range(len(data_validation)):
        if list[i] == data_validation["price_range"][i]:
            count += 1

    return list, count/len(data_validation)

hasil, a = predict(Posterior, categorical_data_validation)
print(hasil)
print(a)

[2, 2, 3, 0, 3, 1, 3, 0, 3, 1, 2, 2, 3, 0, 3, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 2, 0, 2, 1, 3, 3, 1, 2, 3, 1, 3, 1, 1, 0, 2, 0, 2, 2, 1, 1, 3, 1, 3, 1, 2, 3, 0, 1, 3, 2, 3, 3, 1, 1, 3, 3, 0, 3, 2, 2, 2, 2, 2, 2, 3, 1, 0, 1, 2, 0, 2, 1, 0, 3, 3, 0, 2, 3, 0, 3, 3, 0, 3, 1, 1, 0, 2, 1, 0, 3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2, 3, 1, 0, 3, 1, 3, 0, 2, 2, 3, 2, 1, 2, 2, 1, 3, 3, 1, 0, 0, 3, 0, 0, 1, 3, 0, 1, 3, 3, 1, 2, 2, 1, 2, 1, 2, 3, 1, 0, 2, 1, 0, 3, 0, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 0, 2, 3, 1, 0, 1, 1, 1, 3, 0, 2, 0, 3, 1, 1, 2, 0, 2, 0, 3, 1, 0, 3, 2, 0, 1, 2, 0, 1, 3, 3, 1, 1, 2, 2, 3, 2, 3, 3, 3, 0, 3, 0, 2, 3, 1, 2, 3, 2, 0, 0, 2, 2, 1, 3, 3, 0, 2, 0, 3, 0, 2, 1, 2, 2, 1, 3, 0, 1, 3, 3, 3, 0, 0, 1, 1, 3, 3, 1, 2, 0, 2, 1, 2, 3, 3, 0, 2, 1, 1, 1, 2, 0, 2, 2, 3, 3, 0, 1, 3, 3, 0, 1, 2, 1, 1, 1, 0, 3, 3, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 3, 0, 3, 0, 1, 0, 1, 3, 2, 2, 0, 2, 1, 0, 1, 1, 1, 2, 0, 3, 1, 1, 0, 2, 3, 1, 3, 1, 3, 1, 2, 2, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 3, 2, 3, 

In [35]:
import pandas as pd
import numpy as np
import pickle

class NaiveBayes:
    def __init__(self):
        self.data_train = pd.read_csv("../../data/data_train.csv")
        #Memilih kolom yang paling berpengaruh pada penilaian
        self.data_train = self.data_train[["battery_power", "mobile_wt", "px_height", "px_width", "ram", "price_range"]]
        #Mengubah kolom dari nilai numerik menjadi nilai kategorikal
        self.categorical_data_train = self.categorical(self.data_train)

    def categorical(self, dataset):
        batas = [[864.75, 1219.9, 1655.75], [111.0, 143.0, 172.0], [307.0, 576.5, 928.5], [874.0, 1249.0, 1639.75], [1220.75, 2286.0, 3114.5]]
        nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
        for i in range(len(batas)):
            dataset.loc[dataset[nama[i]] <= batas[i][0], nama[i]] = 0
            dataset.loc[(dataset[nama[i]] > batas[i][0]) & (dataset[nama[i]] <= batas[i][1]), nama[i]] = 1
            dataset.loc[(dataset[nama[i]] > batas[i][1]) & (dataset[nama[i]] <= batas[i][2]), nama[i]] = 2
            dataset.loc[dataset[nama[i]] > batas[i][2], nama[i]] = 3
        return dataset
    
    def p_vj(self):
        list = []
        for i in np.unique(self.data_train["price_range"]):
            count = sum(self.data_train["price_range"] == i)
            list.append(count/len(self.data_train))
        return list

    def p_ai_vj(self):
        nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
        list = []
        for i in range(len(nama)):
            kolom = []
            for j in np.unique(self.data_train["price_range"]):
                matrix = []
                count = sum((self.data_train["price_range"] == j) & (self.data_train[nama[i]] == 0))
                matrix.append(count/sum(self.data_train["price_range"] == j))
                count = sum((self.data_train["price_range"] == j) & (self.data_train[nama[i]] == 1))
                matrix.append(count/sum(self.data_train["price_range"] == j))
                count = sum((self.data_train["price_range"] == j) & (self.data_train[nama[i]] == 2))
                matrix.append(count/sum(self.data_train["price_range"] == j))
                count = sum((self.data_train["price_range"] == j) & (self.data_train[nama[i]] == 3))
                matrix.append(count/sum(self.data_train["price_range"] == j))
                kolom.append(matrix)
            list.append(kolom)    
        return list
    
    def posterior(self, data_validation):
        prior = self.p_vj()
        likelihood = self.p_ai_vj()
        data_validation = self.categorical(data_validation)
        nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
        list = []
        for i in range(len(data_validation)):
            matrix = []
            for j in range(len(prior)):
                matrix.append(prior[j])
                for k in range(len(nama)):
                    matrix[j] = matrix[j] * likelihood[k][j][data_validation[nama[k]][i]]

            list.append(matrix)
        return list
    
    def predict(self, data_validation):
        Posterior = self.posterior(data_validation)

        list = []
        for i in range(len(Posterior)):
            list.append(np.argmax(Posterior[i]))
        count = 0
        for i in range(len(data_validation)):
            if list[i] == data_validation["price_range"][i]:
                count += 1
        return list, count/len(data_validation)
    
    def get_result(self, data_val, index):
        posterior = self.posterior(data_val)
        list = []
        for i in range(len(Posterior)):
            list.append(np.argmax(Posterior[i]))

        return list[index]
    
    def save_model(self, filename):
        pickle.dump(self, open("../model/"+ filename, 'wb'))

categorical_data_validation = pd.read_csv("../../data/data_validation.csv")
model = NaiveBayes()
hasil, persentase = model.predict(categorical_data_validation)
coba = model.get_result(categorical_data_validation, 2)
model.save_model("NB_model.pkl")

[2, 2, 3, 0, 3, 1, 3, 0, 3, 1, 2, 2, 3, 0, 3, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 2, 0, 2, 1, 3, 3, 1, 2, 3, 1, 3, 1, 1, 0, 2, 0, 2, 2, 1, 1, 3, 1, 3, 1, 2, 3, 0, 1, 3, 2, 3, 3, 1, 1, 3, 3, 0, 3, 2, 2, 2, 2, 2, 2, 3, 1, 0, 1, 2, 0, 2, 1, 0, 3, 3, 0, 2, 3, 0, 3, 3, 0, 3, 1, 1, 0, 2, 1, 0, 3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2, 3, 1, 0, 3, 1, 3, 0, 2, 2, 3, 2, 1, 2, 2, 1, 3, 3, 1, 0, 0, 3, 0, 0, 1, 3, 0, 1, 3, 3, 1, 2, 2, 1, 2, 1, 2, 3, 1, 0, 2, 1, 0, 3, 0, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 0, 2, 3, 1, 0, 1, 1, 1, 3, 0, 2, 0, 3, 1, 1, 2, 0, 2, 0, 3, 1, 0, 3, 2, 0, 1, 2, 0, 1, 3, 3, 1, 1, 2, 2, 3, 2, 3, 3, 3, 0, 3, 0, 2, 3, 1, 2, 3, 2, 0, 0, 2, 2, 1, 3, 3, 0, 2, 0, 3, 0, 2, 1, 2, 2, 1, 3, 0, 1, 3, 3, 3, 0, 0, 1, 1, 3, 3, 1, 2, 0, 2, 1, 2, 3, 3, 0, 2, 1, 1, 1, 2, 0, 2, 2, 3, 3, 0, 1, 3, 3, 0, 1, 2, 1, 1, 1, 0, 3, 3, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 3, 0, 3, 0, 1, 0, 1, 3, 2, 2, 0, 2, 1, 0, 1, 1, 1, 2, 0, 3, 1, 1, 0, 2, 3, 1, 3, 1, 3, 1, 2, 2, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 3, 2, 3, 

##  Load Model

In [36]:
loaded_model = pickle.load(open("../model/NB_model.pkl", 'rb'))
data_validation = pd.read_csv("../../data/data_validation.csv")
hasil, persentase = loaded_model.predict(data_validation)
coba = loaded_model.get_result(data_validation, 2)

0.7333333333333333
3
